# Mixture-of-Agents in 46 lines of code

![moa-explained](../resources/moa-explained.jpg)

In [ ]:
%pip install langchain-openai langchain-anthropic langchain-google-genai langchain getpass nest_asyncio

In [1]:
import asyncio
from langchain_openai.chat_models import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("ANTHROPIC_API_KEY")

In [6]:
user_prompt = "What are 3 fun things to do in SF?"
reference_models = [
    ChatOpenAI(model='gpt-4o-mini'),
    ChatAnthropic(model="claude-3-5-sonnet-20240620"),
    ChatGoogleGenerativeAI(model="gemini-2.0-flash", api_key=os.environ["GEMINI_API_KEY"])
]
aggregator_model = ChatOpenAI(model='gpt-4o-mini')
aggreagator_system_prompt = """You have been provided with a set of responses from various open-source models to the latest user query. Your task is to synthesize these responses into a single, high-quality response. It is crucial to critically evaluate the information provided in these responses, recognizing that some of it may be biased or incorrect. Your response should not simply replicate the given answers but should offer a refined, accurate, and comprehensive reply to the instruction. Ensure your response is well-structured, coherent, and adheres to the highest standards of accuracy and reliability.

Responses from models:"""

async def run_llm(model):
    """Run a single LLM call with a reference model."""
    for sleep_time in [1, 2, 4]:
        try:
            response = await model.ainvoke([HumanMessage(content=user_prompt)])
            return response.content
        except Exception as e:
            print(f"Error with {model.__class__.__name__}: {e}")
            await asyncio.sleep(sleep_time)
    return "Failed to get response"

In [8]:
import nest_asyncio
nest_asyncio.apply()

async def main():
    results = await asyncio.gather(*[run_llm(model) for model in reference_models])

    final_response = await aggregator_model.ainvoke(
        [
            SystemMessage(
                content=aggreagator_system_prompt
                + "\n"
                + "\n".join(
                    [f"{i+1}. {str(element)}" for i, element in enumerate(results)]
                )
            ),
            HumanMessage(content=user_prompt),
        ]
    )

    print(final_response.content)

await main()

San Francisco is a vibrant city with a wide array of activities and attractions. Here are three fun things to do while visiting:

1. **Explore Golden Gate Park**: This expansive urban park is home to numerous attractions that cater to various interests. You can visit the serene Japanese Tea Garden, explore the California Academy of Sciences, which includes a natural history museum, an aquarium, and a planetarium, or simply enjoy a leisurely bike ride or paddle boat rental on Stow Lake. The park's beautiful landscapes make it an ideal spot for relaxation and outdoor activities.

2. **Visit Alcatraz Island**: A trip to San Francisco is incomplete without visiting the historic Alcatraz Island. Take a ferry from the Ferry Building or Fisherman's Wharf to explore the former prison. The audio tour features compelling narratives from former inmates and guards, providing a fascinating glimpse into the island's history. Be sure to book your tickets in advance, as they tend to sell out quickly, 